In [9]:
import json

In [10]:
import requests
import time
import re
from random import getrandbits
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.sneakersnstuff.com/fr/904/sneakers-homme'
headers = {'User-Agent':
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

def get_page(url):
        email = 'youremail+{}@gmail.com'.format(getrandbits(40))
        payload ={
            'form[email:Fi1mS3bZn3VF]': email,
            'form[landed_at]': '1500901708', # don't change 
            'form[language]': 'en', # don't change
            'form[terms:QS4P6gmSVp8s]': '1', # don't change
            'form[textfield:Boa0BNN75u6R]': 'Amsterdam', # change to ur city
            'form[textfield:EKZm78mYLBbE]': 'Streetname 2', # change to ur streetname + number
            'form[textfield:IIdtgeMwp8Gp]': 'Fock', # first name
            'form[textfield:MQfwQiem4TEw]': '1111AA', # zip code
            'form[textfield:bHSS1rapvhha]': 'NikeTalk', # Last name
            'form[textfield:n3yXjbpstFmJ]': 'None', # change to ur state, if EU set: None
            'form[textfield:u9og65Z22NNB]': 'The Netherlands', # change to ur country
            'form[token]': 'e5455f13bd09203c7f5f0b433129e116$2y$11$e2dJZC0zIXZQK1pxbSZbL.LzJE2QS5SFRX2Z7Lk/oL/j6JicEpiXS', # don't change
            'form[yes-no:warE3SilldbG]': '1', # don't change
                        }
        resp = requests.post(url, data=payload, headers=headers)
        if any(re.findall(r'succes', str(resp.text), re.IGNORECASE)):
            return resp
        else:
            print(time.strftime("%H:%M:%S") + " / Could not enter!")

In [11]:
get_page("https://www.sneakersnstuff.com/fr/product/65586/adidas-sl-72-rs")

<Response [403]>

In [12]:
page = get_page(url)
soup = BeautifulSoup(page.text, 'html.parser')
nb_pages = int(soup.find('span', {'class': 'pagination__total'}).text)

all_elmts = []
for i in range(1, nb_pages + 1):
    print("Fetching page : " + str(i))
    page = get_page(url + "/" + str(i))
    soup = BeautifulSoup(page.text, 'html.parser')
    articles = soup.find("div", {"class": "product-list"}).find_all("article")
    for article in articles:
        elmt = {}
        if article.find("span", {"class" : "card__brand"}) is not None:
            elmt["brand"] = article.find("span", {"class" : "card__brand"}).text
        elmt["name"] = article.find("strong", {"class": "card__name"}).text
        elmt["price"] = article.find("span", {"class" : "price__current"}).text.replace("\n","").replace(" EUR", "")
        elmt["available"] = article.find("div", {"class" : "card__availability"}).text.replace("\n","")
        elmt["url"] = "https://www.sneakersnstuff.com" + article.find("a", {"class" : "card__link"})["href"]
        elmt["img"] = "https://www.sneakersnstuff.com" + article.find("img")["src"]
        elmt["color"] = json.loads(article.attrs["data-gtm-list-product"])["variant"]
        all_elmts.append(elmt)

df = pd.DataFrame(all_elmts)

Fetching page : 1
Fetching page : 2
Fetching page : 3
Fetching page : 4
Fetching page : 5
Fetching page : 6
Fetching page : 7
Fetching page : 8
Fetching page : 9
Fetching page : 10
Fetching page : 11
Fetching page : 12
Fetching page : 13
Fetching page : 14
Fetching page : 15
Fetching page : 16
Fetching page : 17
Fetching page : 18
Fetching page : 19
Fetching page : 20
Fetching page : 21
Fetching page : 22
Fetching page : 23
Fetching page : 24
Fetching page : 25


In [13]:
all_elmts[0]

{'brand': 'Jordan Brand',
 'name': 'Air Jordan 11 Retro',
 'price': '210',
 'available': 'InStock',
 'url': 'https://www.sneakersnstuff.com/fr/product/64072/jordan-brand-air-jordan-11-retro',
 'img': 'https://www.sneakersnstuff.com/images/404854/card_xsmall.jpg',
 'color': 'White'}

In [14]:
df.to_csv("sneakersnstuff.csv", sep=";", index=False)

In [16]:
df["color"].unique()

array(['White', 'Brown', 'Orange', 'Grey', 'Blue', 'Red', 'Yellow',
       'Black', 'Green', 'Purple', 'Multi', 'Silver', 'Pink', None,
       'Gold', ''], dtype=object)

# Just for testing

In [15]:
import numpy as np

def pr_fiabilite(x, prix_fiable, pr_acceptable = 31):
    ecart_type = pr_acceptable / 100 * prix_fiable
    return np.exp( - (x - prix_fiable)**2 / (2 * ecart_type**2)) * 100

pr_fiabilite(220, 130)

8.26034223387487